In [1]:
import cv2
import re
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from time import sleep
from tqdm import tqdm

In [2]:
SSD = ['SSD6']
DATE = ['20230312', '20230314']
SESSION = ['SM_Lek1']
DRONE = ['P2D3', 'P2D4']

frame_range = 10
max_radius = 10
codec = 'mp4v'

In [3]:
for ssd in SSD:
    for date in DATE:
        for session in SESSION:
            for drone in DRONE:
                internal_path = '/' + ssd + '/' + date + '/' + session + '/' + drone
                video_name = date + '_' + session + '_' + drone
                
                video = glob.glob('/media/vsridhar/T7/Work/Data/BlackbuckAnnotations/P2/original/' + video_name + '_*.MP4')
                if video:
                    predictions = glob.glob('/home/vsridhar/DATA/DeepLabCut/projects/05_TerritoryDetectionP2-Vivek-2023-07-10/output' + internal_path + '/' + video_name + '_' + video[0][-12:-4] + '*full.pickle')
                    
                    data = pd.read_pickle(predictions[0])
                    frames = list(data.keys())[1:]
                    
                    for idx,frame in enumerate(frames):
                        fr = np.repeat(int(re.findall(r'\d+', frame)[0]),len(data[frame]['coordinates'][0][0]))
                        pos_x = data[frame]['coordinates'][0][0][:,0]
                        pos_y = data[frame]['coordinates'][0][0][:,1]
                        territory_id = np.arange(0,len(data[frame]['coordinates'][0][0]))

                        tmp = pd.DataFrame(np.array([fr,pos_x,pos_y,territory_id]).T, columns=['frame', 'pos_x', 'pos_y', 'territory_id'])

                        if idx != 0:
                            df = pd.concat((df, tmp), axis=0)
                        else:
                            df = tmp

                    df['hue'] = (df['territory_id']*4).astype(np.int64)
                    
                    input_vidpath = video[0]
                    output_vidpath = '/home/vsridhar/DATA/DeepLabCut/projects/05_TerritoryDetectionP2-Vivek-2023-07-10/output/' + video_name + '_' + video[0][-12:-4] + '_output.MP4'
                    
                    ## Open video
                    cap = cv2.VideoCapture(input_vidpath)
                    if cap.isOpened() == False:
                        sys.exit('Video file cannot be read! Please check input_vidpath to ensure it is correctly pointing to the video file')

                    ## Video writer class to output video with contour and centroid of tracked object(s)
                    # make sure the frame size matches size of array 'final'
                    fourcc = cv2.VideoWriter_fourcc(*codec)
                    output_framesize = (int(cap.read()[1].shape[1]),int(cap.read()[1].shape[0]))
                    out = cv2.VideoWriter(filename = output_vidpath, fourcc = fourcc, fps = 30.0, frameSize = output_framesize, isColor = True)

                    last = 0

                    while True:
                        # Capture frame-by-frame
                        ret, frame = cap.read()

                        this = cap.get(1)

                        tmp = df[(df['frame'] > this-frame_range) & (df['frame'] <= this)]
                        tmp = tmp.reset_index()
                        if ret == True:
                            # Preprocess the image for background subtraction
                            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
                            thresh = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                            thresh = cv2.adaptiveThreshold(thresh, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 501, 20)

                            for i in range(0,len(tmp)):
                                x = int(tmp.loc[i,'pos_x']*3)
                                y = int(tmp.loc[i,'pos_y']*3)
                                r = int(max_radius - (this - tmp.loc[i,'frame'])//(frame_range/max_radius))
                                c = (int(tmp.loc[i,'hue']),255,255)
                                cv2.circle(frame, (x,y), r, c, -1, cv2.LINE_AA)

                            frame = cv2.cvtColor(frame, cv2.COLOR_HSV2BGR)

                            # Display the resulting frame
                            out.write(frame)
#                             cv2.imshow('frame', frame)
                            if cv2.waitKey(1) == 27:
                                break

                        if last >= this:
                            break

                        last = this
                        
                        if this%100 == 0:
                            print(date, session, drone, this)

                    ## When everything done, release the capture
                    cap.release()
                    out.release()
                    cv2.destroyAllWindows()
                    cv2.waitKey(1)

20230312 SM_Lek1 P2D3 100.0
20230312 SM_Lek1 P2D3 200.0
20230312 SM_Lek1 P2D3 300.0
20230312 SM_Lek1 P2D3 400.0
20230312 SM_Lek1 P2D3 500.0
20230312 SM_Lek1 P2D3 600.0
20230312 SM_Lek1 P2D3 700.0
20230312 SM_Lek1 P2D3 800.0
20230312 SM_Lek1 P2D3 900.0
20230312 SM_Lek1 P2D3 1000.0
20230312 SM_Lek1 P2D3 1100.0
20230312 SM_Lek1 P2D3 1200.0
20230312 SM_Lek1 P2D3 1300.0
20230312 SM_Lek1 P2D3 1400.0
20230312 SM_Lek1 P2D3 1500.0
20230312 SM_Lek1 P2D3 1600.0
20230312 SM_Lek1 P2D3 1700.0
20230312 SM_Lek1 P2D3 1800.0
20230312 SM_Lek1 P2D3 1900.0
20230312 SM_Lek1 P2D3 2000.0
20230312 SM_Lek1 P2D3 2100.0
20230312 SM_Lek1 P2D3 2200.0
20230312 SM_Lek1 P2D3 2300.0
20230312 SM_Lek1 P2D3 2400.0
20230312 SM_Lek1 P2D3 2500.0
20230312 SM_Lek1 P2D3 2600.0
20230312 SM_Lek1 P2D3 2700.0
20230312 SM_Lek1 P2D3 2800.0
20230312 SM_Lek1 P2D3 2900.0
20230312 SM_Lek1 P2D3 3000.0
20230312 SM_Lek1 P2D3 3100.0
20230312 SM_Lek1 P2D3 3200.0
20230312 SM_Lek1 P2D3 3300.0
20230312 SM_Lek1 P2D3 3400.0
20230312 SM_Lek1 P2D3 3